In [317]:
# copy & paste this cell in new notebook 
# to have all the og data, and also the modified df, X (feature vars) and y (target var)

# Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import f1_score

# Data, df, X, y
data = pd.read_csv('../../src/data/syriatel_customer_churn.csv')
df = data.drop('phone number', axis=1)
X = df.drop('churn', axis=1)
y = df['churn']

## Importing Data

In [293]:
data = pd.read_csv('../../src/data/syriatel_customer_churn.csv')

data.head()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


## EDA

In [294]:
# checking og data.info()

data.info()

# no n/as

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   state                   3333 non-null   object 
 1   account length          3333 non-null   int64  
 2   area code               3333 non-null   int64  
 3   phone number            3333 non-null   object 
 4   international plan      3333 non-null   object 
 5   voice mail plan         3333 non-null   object 
 6   number vmail messages   3333 non-null   int64  
 7   total day minutes       3333 non-null   float64
 8   total day calls         3333 non-null   int64  
 9   total day charge        3333 non-null   float64
 10  total eve minutes       3333 non-null   float64
 11  total eve calls         3333 non-null   int64  
 12  total eve charge        3333 non-null   float64
 13  total night minutes     3333 non-null   float64
 14  total night calls       3333 non-null   

In [295]:
# checking for dupicates

data.duplicated().value_counts()

# no duplicates

False    3333
dtype: int64

In [296]:
# checking target balance

data.churn.value_counts(normalize=True)

# **LOOKS LIKE CLASS IMBALACE** - check if this needs to be addressed, and how.

False    0.855086
True     0.144914
Name: churn, dtype: float64

In [297]:
# Checking 'phone number' col

data['phone number'].value_counts()

# dropping this since all unique values

360-3126    1
394-8086    1
416-1845    1
361-9839    1
344-5973    1
           ..
400-4344    1
334-4354    1
364-3567    1
337-3868    1
335-4584    1
Name: phone number, Length: 3333, dtype: int64

In [315]:
# Dropping phone number column, 
# and assigning modified dataframe to new variable: df; to preserve og dataframe

df = data.drop('phone number', axis=1)

In [299]:
# checking df.info() to confirm drop

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   state                   3333 non-null   object 
 1   account length          3333 non-null   int64  
 2   area code               3333 non-null   int64  
 3   international plan      3333 non-null   object 
 4   voice mail plan         3333 non-null   object 
 5   number vmail messages   3333 non-null   int64  
 6   total day minutes       3333 non-null   float64
 7   total day calls         3333 non-null   int64  
 8   total day charge        3333 non-null   float64
 9   total eve minutes       3333 non-null   float64
 10  total eve calls         3333 non-null   int64  
 11  total eve charge        3333 non-null   float64
 12  total night minutes     3333 non-null   float64
 13  total night calls       3333 non-null   int64  
 14  total night charge      3333 non-null   

In [300]:
# checking area code value counts

df['area code'].value_counts()

# will keep this.

415    1655
510     840
408     838
Name: area code, dtype: int64

In [301]:
# splitting feature variables and target variable

X = df.drop('churn', axis=1)
y = df['churn']

In [302]:
# Checking dtypes of our features- to see which cols to ohe, and which cols to scale

X.dtypes.value_counts()

float64    8
int64      8
object     3
dtype: int64

In [303]:
# train-test-split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [304]:
# checking shape of split data sets

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(2499, 19) (2499,)
(834, 19) (834,)


## Preprocessing

#### Train data

In [305]:
# splitting train data into numerical and categoricals columns

X_tr_num = X_train.select_dtypes(exclude='object') # numerical cols
X_tr_cat = X_train.select_dtypes(include='object') # categorical cols

In [306]:
# ohe'ing train categorical cols 

ohe = OneHotEncoder(sparse= False, drop= 'first')

X_tr_ohe = pd.DataFrame(ohe.fit_transform(X_tr_cat), 
                        index= X_tr_cat.index, 
                        columns=ohe.get_feature_names(X_tr_cat.columns))
print(X_tr_ohe.shape)


# standardizing train numerical cols

ss = StandardScaler()

X_tr_sc = pd.DataFrame(ss.fit_transform(X_tr_num), 
                       index= X_tr_num.index, 
                       columns= X_tr_num.columns)
print(X_tr_sc.shape)

(2499, 52)
(2499, 16)


In [307]:
# joining train num and ohe cat cols together

X_tr_final = X_tr_sc.join(X_tr_ohe)
X_tr_final.head()

,account length,area code,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,...,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY,international plan_yes,voice mail plan_yes
367,-1.404508,-0.512381,-0.584700,-1.883677,1.330852,-1.884170,1.037727,0.401340,1.037905,1.069609,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3103,0.366388,-0.512381,-0.584700,0.294083,0.529165,0.293703,0.516178,0.401340,0.517286,2.214376,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
549,0.518179,-0.679077,1.685101,1.056392,-1.875896,1.056666,0.093407,0.849774,0.094283,-0.077125,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2531,2.010792,-0.512381,-0.584700,-0.679156,1.681590,-0.679320,-0.402459,0.650470,-0.403094,-0.322994,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2378,0.290493,1.749923,-0.584700,0.484660,1.080325,0.484172,-0.718549,-0.296224,-0.719184,-1.186487,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Modeling

#### Preproessing Test Data (same as we did on the train data)

In [308]:
# splitting test data into numerical and categoricals columns

X_te_num = X_test.select_dtypes(exclude='object') # numerical cols
X_te_cat = X_test.select_dtypes(include='object') # categorical cols

In [309]:
# ohe'ing test categorical cols 

X_te_ohe = pd.DataFrame(ohe.transform(X_te_cat), 
                        index= X_te_cat.index, 
                        columns=ohe.get_feature_names(X_te_cat.columns))
print(X_te_ohe.shape)


# standardizing test numerical cols

X_te_sc = pd.DataFrame(ss.transform(X_te_num), 
                       index= X_te_num.index, 
                       columns= X_te_num.columns)
print(X_te_sc.shape)

(834, 52)
(834, 16)


In [310]:
# joining test num and ohe cat cols together

X_te_final = X_te_sc.join(X_te_ohe)
X_te_final.head()

,account length,area code,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,...,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY,international plan_yes,voice mail plan_yes
438,0.315791,1.749923,-0.5847,-0.462675,-0.372733,-0.462730,2.562862,0.301688,2.562574,-0.220713,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2674,-0.847941,-0.512381,-0.5847,-1.311946,0.829797,-1.311676,0.326524,1.198556,0.326702,-0.240382,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1345,-0.063687,-0.512381,-0.5847,-3.330584,-5.032539,-3.330643,-0.815352,1.497512,-0.814476,-0.659343,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1957,1.175941,-0.679077,-0.5847,0.606778,-1.074209,0.607160,0.063774,-0.445702,0.064068,-0.873741,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2148,-0.114284,-0.679077,-0.5847,-0.666204,0.078216,-0.666259,0.470740,-1.342571,0.470802,0.532630,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


### Logistic Regression

#### lg 1

In [311]:
# lg 1 (random_state=42)
# instantiating logistic regression instance and fitting final train data to model. 
# calc'ed f1 score of lg1 on final test data.

lg1 = LogisticRegression(random_state=42)

lg1.fit(X_tr_final, y_train)

lg1_y_pred = lg1.predict(X_te_final)

f1_score(y_test, lg1_y_pred)

0.29585798816568043

In [312]:
# accuracy score of lg1

lg1.score(X_te_final, y_test)

0.8573141486810552

In [313]:
# cross val score- f1 score & accuracy- for both train X/y and test X/y.

print(f"train f1: {cross_val_score(LogisticRegression(random_state=42), X_tr_final, y_train, cv= 5, scoring= 'f1').mean()},", 
      f"train accuracy: {cross_val_score(LogisticRegression(random_state=42), X_tr_final, y_train, cv= 5, scoring= 'accuracy').mean()},")

print(f"test f1: {cross_val_score(LogisticRegression(random_state=42), X_te_final, y_test, cv= 5, scoring= 'f1').mean()},", 
    f"test accuracy: {cross_val_score(LogisticRegression(random_state=42), X_te_final, y_test, cv= 5, scoring= 'accuracy').mean()}")

# Doesn't seem like we're overfitting

train f1: 0.30778284332902006, train accuracy: 0.861543887775551,
test f1: 0.32727757661968193, test accuracy: 0.8573046677728879


In [314]:
# checking coefs of lg1

np.set_printoptions(suppress=True) # removing scientific notation

lg1.coef_

array([[ 0.03422827,  0.07489295,  0.04076176,  0.315816  ,  0.01535521,
         0.31831366,  0.1780134 ,  0.02981691,  0.16884381,  0.07823876,
        -0.0039107 ,  0.07054775,  0.11178518, -0.23549467,  0.13845564,
         0.68296919, -0.64746282,  0.41792704, -0.48430557,  0.46197577,
        -0.2885145 ,  0.07551281, -0.0017391 , -0.15194732, -0.11994506,
         0.21849399, -0.52443654, -0.73027625, -0.07054295, -0.69144723,
         0.11109838,  0.31083592,  0.23553295, -0.26489868,  0.21547413,
         0.4234435 ,  0.55347512,  0.42596333,  0.39281097, -0.18461133,
         0.48841423,  0.91446951,  0.16153352, -1.06253891,  0.09823615,
         0.12964936,  0.63357835, -0.1053758 ,  0.57900793,  0.29930965,
         0.00803035, -0.13135712,  0.30654225,  0.53511023, -0.45028458,
         0.66295362, -0.25027413, -0.25532122,  0.79290588, -0.05793457,
        -0.59147197, -0.936388  ,  0.46065258, -0.25750093, -0.07611292,
        -0.23023528,  2.13903853, -0.9704051 ]])

In [231]:
# what do these coefs mean? should we do a heat map? a confusion matrix?
# look into feature selection for log reg, and also param hypertuning